In [2]:
import os
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader,WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_community.document_loaders import PyPDFDirectoryLoader
import time
import pickle

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
def get_all_links_from_index(index_url):
    response = requests.get(index_url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    base_url = "https://tugraph-db.readthedocs.io/zh-cn/latest/"
    links = []

    # 查找所有 <a> 标签，提取 href 属性中的子页面链接
    for a_tag in soup.find_all("a", href=True):
        href = a_tag['href']
        # 如果链接是相对路径的子页面，转换为完整URL
        if href.startswith('/') or not href.startswith('https'):
            full_url = base_url + href if not href.startswith('/') else base_url + href[1:]
            links.append(full_url)

    # 过滤掉重复的链接
    links = list(set(links))

    return links

In [5]:
# Step 2: 从主索引页面递归提取所有链接
index_url = "https://tugraph-db.readthedocs.io/zh-cn/latest/index.html"
all_links = get_all_links_from_index(index_url)
print(len(all_links))

index_url = "https://tugraph-db.readthedocs.io/zh-cn/latest/1.guide.html"
all_links = list(set(get_all_links_from_index(index_url) + all_links))
print(len(all_links))

# Step 3: 合并手动链接和索引页面获取的链接
manual_links = [
    "https://tugraph-db.readthedocs.io/zh-cn/v4.3.2/5.installation%26running/8.high-availability-mode.html",
    "https://www.oceanbase.com/docs/enterprise-tugraph-doc-cn-10000000001669466",
    "https://www.oceanbase.com/docs/enterprise-tugraph-doc-cn-10000000001032011",
    "https://www.oceanbase.com/docs/enterprise-tugraph-doc-cn-10000000001667857",
    "https://www.oceanbase.com/docs/enterprise-tugraph-doc-cn-10000000001102367",
    "https://www.modb.pro/db/656598",
    "https://tugraph-db.readthedocs.io/zh-cn/latest/7.client-tools/8.rpc-api.html",
    "https://www.modb.pro/db/1782579391289651200",
    "https://tugraph-db.readthedocs.io/zh-cn/v3.5.1/5.developer-manual/2.running/2.tugraph-running.html",
    "https://tugraph-db-lipanpan.readthedocs.io/zh-cn/latest/6.utility-tools/6.tugraph-cli.html",
    "https://tugraph-db-lipanpan.readthedocs.io/zh-cn/latest/8.query/3.vector_index.html",
    "https://www.modb.pro/db/483644",
    "https://tugraph-db.readthedocs.io/zh-cn/latest/6.utility-tools/1.data-import.html",
    "https://tugraph-db.readthedocs.io/zh-cn/latest/4.user-guide/1.tugraph-browser.html",
    "https://blog.csdn.net/gitblog_00098/article/details/142606629",
    "https://blog.csdn.net/gitblog_00098/article/details/142606629",
    "https://www.modb.pro/db/628435",
    "https://tugraph-db.readthedocs.io/zh-cn/v4.1.0/5.developer-manual/2.running/2.tugraph-running.html",
    "https://help.aliyun.com/zh/compute-nest/use-cases/tugraph-service-instance-deployment-documentation",
    "https://www.modb.pro/db/656572",
    "https://tugraph-db.readthedocs.io/zh-cn/latest/2.introduction/4.schema.html",
    "https://tugraph-db.readthedocs.io/zh-cn/latest/9.olap&procedure/",
]

# 合并手动链接和从索引页获取的链接
all_links = list(set(manual_links + all_links))  # 去重后合并
print(len(all_links))

# 读取successful_urls.txt中的所有链接
with open("successful_urls.txt", "r") as file:
    # 假设每一行是一个链接，strip()去除换行符
    successful_urls = [line.strip() for line in file.readlines()]

# 合并并去重 manual_links, all_links, 和 successful_urls
all_links = list(set(manual_links + all_links + successful_urls))

# 打印去重后的链接数量
print(len(all_links))

94
108
125
242


In [6]:
# Step 3: 将提取到的子页面URL传递给 WebBaseLoader 进行内容加载
loader = WebBaseLoader(all_links)
documents = loader.load()

/home/pigkiller/anaconda3/envs/dl/lib/python3.8/site-packages/langchain_community/document_loaders/web_base.py:316: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(html_doc.text, parser, **(bs_kwargs or {}))


In [7]:
separators = ["\n\n", "\n", "。"]  # 定义分割符
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=512, separators=separators, keep_separator=False)
documents = text_splitter.split_documents(documents)

In [9]:
for doc in documents:
    print(f"Source: {doc.metadata['source']}")
    print(f"Content: {doc.page_content[:500]}...")

Source: https://tugraph-db.readthedocs.io/zh-cn/latest/#tugraph
Content: TuGraph高性能图数据库 — TuGraph  文档


            TuGraph
          


Contents:

文档地图
TuGraph入门
快速上手
可视化操作指南
业务开发指南
安装和运行
实用工具
客户端工具
查询语言
存储过程和分析接口
运维与权限管理
测试与质量保障
贡献者文档
最佳实践
FAQ
联系方式


TuGraph


TuGraph高性能图数据库

 查看页面源码

TuGraph高性能图数据库

Contents:

文档地图
快速上手
开发指南
参与社区贡献
主要仓库
视频中心
TuGraph最新版本


TuGraph入门
什么是图
什么是图数据库
什么是TuGraph
TuGraph图模型说明
产品特点
TuGraph产品架构
功能概览
应用场景
名词解释


快速上手
快速上手
demo示例


可视化操作指南
可视化操作手册
可视化操作手册（旧版）


业务开发指南
连接tugraph-db
子图操作
点类型操作
边类型操作
实时查看当前点边数据量
导入数据
导出数据


安装和运行
环境准备
环境分类
Dock...
Source: https://tugraph-db.readthedocs.io/zh-cn/latest/#tugraph
Content: 实用工具
数据导入
数据导出
备份恢复
数据预热
集群管理
命令行工具
TuGraph-DataX
TuGraph-Explorer
TuGraph-Restful-Server


客户端工具
Python客户端
C++客户端
Java客户端
TuGraph-OGM
Bolt客户端
TuGraph console client
RESTful API
RPC API
RESTful API Legacy


查询语言
Cypher API
ISO GQL
Vector index


存储过程和分析接口
存储过程
分析接口
图学习


运维与权限管理
用户权限
Token使用说明
忘记’admin’密码
运维监控
日志信息


测试与质量保障
单元测试
集成测试


贡献者文档
如何贡献


In [12]:
model_name = "BAAI/bge-large-zh-v1.5"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceBgeEmbeddings(
                model_name=model_name,
                model_kwargs=model_kwargs,
                encode_kwargs=encode_kwargs,
                query_instruction="为这个句子生成表示以用于检索相关文章:"
            )

In [13]:
db = FAISS.from_documents(documents, embedding=embedding)
db.save_local('tu_document_1024_512')

In [14]:
retriever=db.as_retriever( search_kwargs={"k": 3})
print(retriever.get_relevant_documents("RPC 及 HA 服务中，verbose 参数的设置有几个级别？"))

[Document(metadata={'source': 'https://tugraph-db.readthedocs.io/zh-cn/latest/_sources/5.installation&running/7.tugraph-running.md.txt'}, page_content='| enable_rpc                   | 布尔值                   | 是否使用 RPC 服务。默认值为 false。                                                                                                                                                           |\n| rpc_port                     | 整型                    | RPC 及 HA 服务所用端口。默认端口为 9090。                                                                                                                                                       |\n| bolt_port                    | 整型                    | Bolt 客户端端口。默认端口为 7687。                                                                                                                                                            |\n| enable_ha                    | 布尔值                   | 是否启动高可用模式。默认值为 false。                                                        

/tmp/ipykernel_25859/1077594208.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  print(retriever.get_relevant_documents("RPC 及 HA 服务中，verbose 参数的设置有几个级别？"))


In [15]:
print(retriever.get_relevant_documents("`FieldData` 类中的函数 `IsReal()` 是用来查询什么类型的数据？"))

[Document(metadata={'source': 'https://tugraph-db.readthedocs.io/zh-cn/latest/_sources/7.client-tools/9.restful-api-legacy.md.txt'}, page_content='其中`fields`为一个数组，其中每个元素定义数据的一列，内容如下：\n\n| 域名     | 说明                                     | 类型                                                                                                |\n| -------- | ---------------------------------------- | --------------------------------------------------------------------------------------------------- |\n| name     | 列名                                     | 字符串                                                                                              |\n| type     | 列数据类型                               | 字符串，有以下类型： int8, int16, int32, int64, float, double, string, date, datetime, binary, bool |\n| optional | 数据是否可以为空（可选，缺省值为 false） | 布尔值                                                                                              |\n\n- **RESPONSE**: 如果成功，返回代码 200。\n\n**Example request.**'), Docum